In [5]:
import pandas as pd 
import numpy as ap
from datetime import timedelta
import numpy as np
from pycaret.regression import *

In [6]:
df_future = pd.read_csv('file_predict\predict_jsps014.h.csv')

In [7]:
last_date = df_future["timestamp"].max()
last_date = pd.to_datetime(last_date)

In [8]:
future_date = pd.date_range(start=last_date + timedelta(days=1), periods=8, freq="D")
future_date

DatetimeIndex(['2025-02-19 11:12:43', '2025-02-20 11:12:43',
               '2025-02-21 11:12:43', '2025-02-22 11:12:43',
               '2025-02-23 11:12:43', '2025-02-24 11:12:43',
               '2025-02-25 11:12:43', '2025-02-26 11:12:43'],
              dtype='datetime64[ns]', freq='D')

In [9]:
future_dates = pd.DataFrame({"timestamp": future_date})
future_dates.head()

,timestamp
0,2025-02-19 11:12:43
1,2025-02-20 11:12:43
2,2025-02-21 11:12:43
3,2025-02-22 11:12:43
4,2025-02-23 11:12:43


In [10]:
Q1_H = df_future["humidity"].quantile(0.25)
Q3_H = df_future["humidity"].quantile(0.75)
IQR_H = Q3_H - Q1_H

lower_bound_H = max(0, Q1_H - 1.5 * IQR_H) 
upper_bound_H = Q3_H + 1.5 * IQR_H 

Q1_TT = df_future["temperature"].quantile(0.25)
Q3_TT = df_future["temperature"].quantile(0.75)
IQR_TT = Q3_TT - Q1_TT

lower_bound_TT = max(0, Q1_TT - 1.5 * IQR_TT) 
upper_bound_TT = Q3_TT + 1.5 * IQR_TT 


In [11]:
future_dates["month"] = future_dates["timestamp"].dt.month
future_dates["dayofweek"] = future_dates["timestamp"].dt.dayofweek
future_dates["weekofyear"] = future_dates["timestamp"].dt.dayofyear
future_dates["day"] = future_dates["timestamp"].dt.day
future_dates["hour"] = future_dates["timestamp"].dt.hour


pm2_5_mean = df_future["temperature"].mean()
pm2_5_std = df_future["temperature"].std()

hum_mean = df_future["humidity"].mean()
hum_std = df_future["humidity"].std()

future_dates["humidity"] = np.random.normal(hum_mean, hum_std, len(future_dates))
future_dates["temperature"] = np.random.normal(pm2_5_mean, pm2_5_std, len(future_dates))

future_dates["humidity"] = future_dates["humidity"].clip(lower=lower_bound_H, upper=upper_bound_H)
future_dates["temperature"] = future_dates["temperature"].clip(lower=lower_bound_TT, upper=upper_bound_TT)
future_dates

,timestamp,month,dayofweek,weekofyear,day,hour,humidity,temperature
0,2025-02-19 11:12:43,2,2,50,19,11,55.420918,31.069383
1,2025-02-20 11:12:43,2,3,51,20,11,63.467029,33.427957
2,2025-02-21 11:12:43,2,4,52,21,11,69.049087,32.437098
3,2025-02-22 11:12:43,2,5,53,22,11,68.343579,29.224303
4,2025-02-23 11:12:43,2,6,54,23,11,53.816693,31.020661
5,2025-02-24 11:12:43,2,0,55,24,11,65.230570,26.737142
6,2025-02-25 11:12:43,2,1,56,25,11,69.637721,32.926846
7,2025-02-26 11:12:43,2,2,57,26,11,66.443788,27.770003


In [12]:
df = pd.concat([df_future,future_dates], ignore_index=True)

In [13]:
df['pm_2_5_lag_12day'] = df['pm_2_5'].shift(336)
df['PM2.5_MA3_prev_12day'] = df['pm_2_5'].shift(336).rolling(window=336).mean()
df['PM2.5_MAX24_prev_12day'] = df['pm_2_5'].shift(336).rolling(window=336).max()
df['PM2.5_MIN24_prev_12day'] = df['pm_2_5'].shift(336).rolling(window=336).min()
df.tail(12)

,Unnamed: 0.1,Unnamed: 0,timestamp,humidity,pm_2_5,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day
23930,23930.0,NaN,2025-02-15 11:12:43,90.161433,NaN,29.851736,2,5,46,15,11,11.445378,20.419579,47.341667,1.133333
23931,23931.0,NaN,2025-02-16 11:12:43,76.656375,NaN,29.331557,2,6,47,16,11,10.735043,20.352135,47.341667,1.133333
23932,23932.0,NaN,2025-02-17 11:12:43,65.767412,NaN,28.004407,2,0,48,17,11,12.066116,20.288331,47.341667,1.133333
23933,23933.0,NaN,2025-02-18 11:12:43,59.402024,NaN,24.984442,2,1,49,18,11,10.974790,20.217553,47.341667,1.133333
23934,NaN,NaN,2025-02-19 11:12:43,55.420918,NaN,31.069383,2,2,50,19,11,10.725000,20.144740,47.341667,1.133333
23935,NaN,NaN,2025-02-20 11:12:43,63.467029,NaN,33.427957,2,3,51,20,11,10.446281,20.068092,47.341667,1.133333
23936,NaN,NaN,2025-02-21 11:12:43,69.049087,NaN,32.437098,2,4,52,21,11,9.983193,19.990430,47.341667,1.133333
23937,NaN,NaN,2025-02-22 11:12:43,68.343579,NaN,29.224303,2,5,53,22,11,10.327586,19.905292,47.341667,1.133333
23938,NaN,NaN,2025-02-23 11:12:43,53.816693,NaN,31.020661,2,6,54,23,11,9.491667,19.799363,47.341667,1.133333
23939,NaN,NaN,2025-02-24 11:12:43,65.230570,NaN,26.737142,2,0,55,24,11,10.733333,19.713071,47.341667,1.133333


In [14]:
df.drop(columns=(["Unnamed: 0"]))

,Unnamed: 0.1,timestamp,humidity,pm_2_5,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day
0,0.0,2022-05-01 17:00:00,74.810899,7.205128,28.220940,5,6,121,1,17,NaN,NaN,NaN,NaN
1,1.0,2022-05-01 18:00:00,73.963971,9.208333,28.551333,5,6,121,1,18,NaN,NaN,NaN,NaN
2,2.0,2022-05-01 19:00:00,74.998299,12.883333,28.158917,5,6,121,1,19,NaN,NaN,NaN,NaN
3,3.0,2022-05-01 20:00:00,76.198977,15.483333,28.019250,5,6,121,1,20,NaN,NaN,NaN,NaN
4,4.0,2022-05-01 21:00:00,78.530695,12.400000,27.682333,5,6,121,1,21,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23937,NaN,2025-02-22 11:12:43,68.343579,NaN,29.224303,2,5,53,22,11,10.327586,19.905292,47.341667,1.133333
23938,NaN,2025-02-23 11:12:43,53.816693,NaN,31.020661,2,6,54,23,11,9.491667,19.799363,47.341667,1.133333
23939,NaN,2025-02-24 11:12:43,65.230570,NaN,26.737142,2,0,55,24,11,10.733333,19.713071,47.341667,1.133333
23940,NaN,2025-02-25 11:12:43,69.637721,NaN,32.926846,2,1,56,25,11,12.258333,19.628324,47.341667,1.133333


In [15]:
df_predict=df.tail(7)
df_predict.drop(columns=(["Unnamed: 0","pm_2_5"]),inplace=True)
df_predict

,Unnamed: 0.1,timestamp,humidity,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day
23935,NaN,2025-02-20 11:12:43,63.467029,33.427957,2,3,51,20,11,10.446281,20.068092,47.341667,1.133333
23936,NaN,2025-02-21 11:12:43,69.049087,32.437098,2,4,52,21,11,9.983193,19.990430,47.341667,1.133333
23937,NaN,2025-02-22 11:12:43,68.343579,29.224303,2,5,53,22,11,10.327586,19.905292,47.341667,1.133333
23938,NaN,2025-02-23 11:12:43,53.816693,31.020661,2,6,54,23,11,9.491667,19.799363,47.341667,1.133333
23939,NaN,2025-02-24 11:12:43,65.230570,26.737142,2,0,55,24,11,10.733333,19.713071,47.341667,1.133333
23940,NaN,2025-02-25 11:12:43,69.637721,32.926846,2,1,56,25,11,12.258333,19.628324,47.341667,1.133333
23941,NaN,2025-02-26 11:12:43,66.443788,27.770003,2,2,57,26,11,10.801653,19.548227,47.341667,1.133333


In [ ]:
loaded_Extra_Trees_Regressors = load_model('pm25_Extra_Trees_Regressor')

Transformation Pipeline and Model Successfully Loaded


In [ ]:
predict = predict_model(loaded_Extra_Trees_Regressors,data=df_predict)
predict

,Unnamed: 0.1,timestamp,humidity,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day,prediction_label
23935,NaN,2025-02-20 11:12:43,63.467030,33.427956,2,3,51,20,11,10.446281,20.068092,47.341667,1.133333,12.943349
23936,NaN,2025-02-21 11:12:43,69.049088,32.437099,2,4,52,21,11,9.983193,19.990431,47.341667,1.133333,15.060965
23937,NaN,2025-02-22 11:12:43,68.343582,29.224302,2,5,53,22,11,10.327586,19.905293,47.341667,1.133333,14.685852
23938,NaN,2025-02-23 11:12:43,53.816692,31.020662,2,6,54,23,11,9.491667,19.799362,47.341667,1.133333,12.980647
23939,NaN,2025-02-24 11:12:43,65.230568,26.737143,2,0,55,24,11,10.733334,19.713072,47.341667,1.133333,17.397739
23940,NaN,2025-02-25 11:12:43,69.637718,32.926846,2,1,56,25,11,12.258333,19.628325,47.341667,1.133333,17.048330
23941,NaN,2025-02-26 11:12:43,66.443787,27.770002,2,2,57,26,11,10.801653,19.548227,47.341667,1.133333,17.749854
